In [13]:
import os
from utils.file_ops import add_filename_timestamp
import shutil
import pandas as pd

In [14]:
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\git\grundfos-express-tools\pipe diameter finder\input files"
file = "Mech GRP foundation for VLS_CUE rev2.xlsx"
filePath = os.path.join(myDir, file)

# Create working copy
completed_dir = r"C:\Users\104092\OneDrive - Grundfos\Documents\git\grundfos-express-tools\pipe diameter finder\output files"
working_copy = os.path.join(completed_dir, add_filename_timestamp(file))
shutil.copyfile(filePath, working_copy) # creates working copy to leave original file untouched

'C:\\Users\\104092\\OneDrive - Grundfos\\Documents\\git\\grundfos-express-tools\\pipe diameter finder\\output files\\2022-08-12_09-55-58 Mech GRP foundation for VLS_CUE rev2.xlsx'

In [15]:
# Read in Excel sheet to a dataframe
sheetname = "Max flow to diameter, Rev1"
data = pd.read_excel(working_copy, sheet_name=sheetname)
data.dropna(axis=1, how='all', inplace=True) # Drops nan columns

In [16]:
branch_data = data[["Max Branch Flow (gpm)","Max Branch Diameter (in.)"]]
branch_data['copy_index'] = branch_data.index

header_data = data[["Max Header Flow (gpm)","Max Header Diameter (in.)"]]
header_data['copy_index'] = header_data.index

reference_df = data[["Flow (gpm)", "Pipe Diameter (in.)"]]
reference_df.set_index("Flow (gpm)", inplace=True)
reference_df.dropna(axis='index', how='any', inplace=True)

C:\Users\104092\AppData\Local\Temp\ipykernel_127004\2757576503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  branch_data['copy_index'] = branch_data.index
C:\Users\104092\AppData\Local\Temp\ipykernel_127004\2757576503.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  header_data['copy_index'] = header_data.index
C:\Users\104092\AppData\Local\Temp\ipykernel_127004\2757576503.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

In [17]:
branch_output_df = pd.merge_asof(branch_data.sort_values('Max Branch Flow (gpm)'), reference_df, left_on="Max Branch Flow (gpm)", right_on="Flow (gpm)", direction='backward')
branch_output_df.sort_values(by=['copy_index'], inplace=True)
header_output_df = pd.merge_asof(header_data.sort_values('Max Header Flow (gpm)'), reference_df, left_on="Max Header Flow (gpm)", right_on="Flow (gpm)", direction='backward')
header_output_df.sort_values(by=['copy_index'], inplace=True)

In [18]:
output_df = pd.merge(branch_output_df, header_output_df, on="copy_index")
output_df.drop(['copy_index'], axis=1, inplace=True)

In [19]:
# Write resulting dataframes to new sheet in PSD with changes.
with pd.ExcelWriter(working_copy, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
    output_df.to_excel(writer, sheet_name="Run 2", index=False)